# DX 704 Week 4 Project

This week's project will test the learning speed of linear contextual bandits compared to unoptimized approaches.
You will start with building a preference data set for evaluation, and then implement different variations of LinUCB and visualize how fast they learn the preferences.


The full project description, a template notebook and supporting code are available on GitHub: [Project 4 Materials](https://github.com/bu-cds-dx704/dx704-project-04).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Rating Data

The file "recipes.tsv" in this repository has information about 100 recipes.
Make a new file "ratings.tsv" with two columns, recipe_slug (from recipes.tsv) and rating.
Populate the rating column with values between 0 and 1 where 0 is the worst and 1 is the best.
You can assign these ratings however you want within that range, but try to make it reflect a consistent set of preferences.
These could be your preferences, or a persona of your choosing (e.g. chocolate lover, bacon-obsessed, or sweet tooth).
Make sure that there are at least 10 ratings of zero and at least 10 ratings of one.


Hint: You may find it more convenient to assign raw ratings from 1 to 5 and then remap them as follows.

`ratings["rating"] = (ratings["rating_raw"] - 1) * 0.25`

Submit "ratings.tsv" in Gradescope.

In [1]:
import pandas as pd, numpy as np

recipes = pd.read_csv("recipes.tsv", sep="\t", dtype=str).fillna("")
text = recipes.apply(lambda r: " ".join(r.values).lower(), axis=1)

rating_raw = np.full(len(recipes), 3.0)
pos2 = text.str.contains(r"\b(chocolate|cocoa|brownie|cookie|cake|dessert|ice cream)\b")
pos1 = text.str_contains = text.str.contains(r"\b(bacon|cheese|butter|cream|caramel|chicken)\b")
neg2 = text.str.contains(r"\b(tofu|tempeh|kale|quinoa|lentil)\b")
neg1 = text.str.contains(r"\b(vegan|salad|raw)\b")

rating_raw += 2*pos2.astype(int) + 1*pos1.astype(int) - 2*neg2.astype(int) - 1*neg1.astype(int)
rating_raw = np.clip(rating_raw, 1, 5)

ratings = pd.DataFrame({"recipe_slug": recipes["recipe_slug"].astype(str)})
ratings["rating"] = (rating_raw - 1.0) * 0.25  # 1->0.0, 5->1.0

z = (ratings["rating"] == 0.0).sum()
if z < 10:
    idx = ratings.sort_values("rating").query("rating > 0").head(10 - z).index
    ratings.loc[idx, "rating"] = 0.0

o = (ratings["rating"] == 1.0).sum()
if o < 10:
    idx = ratings.sort_values("rating", ascending=False).query("rating < 1").head(10 - o).index
    ratings.loc[idx, "rating"] = 1.0


with open("ratings.tsv", "w", newline="") as f:
    ratings.to_csv(f, sep="\t", index=False)


/tmp/ipykernel_5284/3371007521.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  pos2 = text.str.contains(r"\b(chocolate|cocoa|brownie|cookie|cake|dessert|ice cream)\b")
/tmp/ipykernel_5284/3371007521.py:8: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  pos1 = text.str_contains = text.str.contains(r"\b(bacon|cheese|butter|cream|caramel|chicken)\b")
/tmp/ipykernel_5284/3371007521.py:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  neg2 = text.str.contains(r"\b(tofu|tempeh|kale|quinoa|lentil)\b")
/tmp/ipykernel_5284/3371007521.py:10: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  neg1 = text.str.contains(r"\b(vegan|salad|raw)\b")


## Part 2: Construct Model Input

Use your file "ratings.tsv" combined with "recipe-tags.tsv" to create a new file "features.tsv" with a column recipe_slug, a column bias which is hard-coded to one, and a column for each tag that appears in "recipe-tags.tsv".
The tag column in this file should be a 0-1 encoding of the recipe tags for each recipe.
[Pandas reshaping function methods](https://pandas.pydata.org/docs/user_guide/reshaping.html) may be helpful.

The bias column will make later LinUCB calculations easier since it will just be another dimension.

Hint: For later modeling steps, it will be important to have the feature data (inputs) and the rating data (target outputs) in the same order.
It is highly recommended to make sure that "features.tsv" and "ratings.tsv" have the recipe slugs in the same order.

In [2]:
# YOUR CHANGES HERE

import pandas as pd

ratings = pd.read_csv("ratings.tsv", sep="\t", dtype={"recipe_slug": str})
rt = pd.read_csv("recipe-tags.tsv", sep="\t", dtype=str)

if "recipe_slug" not in rt.columns or "tag" not in rt.columns:
    rt = rt.rename(columns={rt.columns[0]: "recipe_slug", rt.columns[1]: "tag"})

tag_matrix = pd.crosstab(rt["recipe_slug"], rt["tag"]).clip(upper=1).astype("int8")

tag_matrix = tag_matrix.reindex(ratings["recipe_slug"], fill_value=0)

features = pd.DataFrame({
    "recipe_slug": ratings["recipe_slug"],
    "bias": 1
}).join(tag_matrix)



with open("features.tsv", "w", newline="", encoding="utf-8") as f:
    features.to_csv(f, sep="\t", index=False)


Submit "features.tsv" in Gradescope.

## Part 3: Linear Preference Model

Use your feature and rating files to build a ridge regression model with ridge regression's regularization parameter $\alpha$ set to 1.


Hint: If you are using scikit-learn modeling classes, you should use `fit_intercept=False` since that intercept value will be redundant with the bias coefficient.

Hint: The estimate component of the bounds should match the previous estimate, so you should be able to just focus on the variance component of the bounds now.

Save the coefficients of this model in a file "model.tsv" with columns "recipe_tag" and "coefficient".
Do not add anything for the `intercept_` attribute of a scikit-learn model; this will be covered by the coefficient for the bias column added in part 2.

Submit "model.tsv" in Gradescope.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge

ratings  = pd.read_csv("ratings.tsv",  sep="\t", dtype={"recipe_slug": str})
features = pd.read_csv("features.tsv", sep="\t", dtype={"recipe_slug": str})

features = features.set_index("recipe_slug").reindex(ratings["recipe_slug"])

features = features.apply(pd.to_numeric, errors="coerce").fillna(0.0)
if "bias" in features.columns:
    features["bias"] = 1.0
else:
    features.insert(0, "bias", 1.0)

X = features.values.astype(np.float32)
y = ratings["rating"].values.astype(np.float32)

alpha = 1.0
model = Ridge(alpha=alpha, fit_intercept=False)
model.fit(X, y)

feature_cols = features.columns.tolist()             
coef = np.asarray(model.coef_).ravel()               

df = pd.DataFrame({"recipe_tag": feature_cols, "coefficient": coef})
with open("model.tsv", "w", newline="", encoding="utf-8") as f:
    df.to_csv(f, sep="\t", index=False)

print(f"Wrote model.tsv with {len(feature_cols)} rows.")


Wrote model.tsv with 297 rows.


## Part 4: Recipe Estimates

Use the recipe model to estimate the score of every recipe.
Save these estimates to a file "estimates.tsv" with columns recipe_slug and score_estimate.

In [ ]:
# YOUR CHANGES HERE


import pandas as pd
import numpy as np

ratings  = pd.read_csv("ratings.tsv",  sep="\t", dtype={"recipe_slug": str})
features = pd.read_csv("features.tsv", sep="\t", dtype={"recipe_slug": str}).set_index("recipe_slug")
model_df = pd.read_csv("model.tsv",    sep="\t")  

features = features.apply(pd.to_numeric, errors="coerce").fillna(0.0)
if "bias" in features.columns:
    features["bias"] = 1.0
else:
    features.insert(0, "bias", 1.0)

cols = model_df["recipe_tag"].tolist()
for c in cols:
    if c not in features.columns:
        features[c] = 0.0

X = features.reindex(columns=cols).reindex(ratings["recipe_slug"]).values.astype(np.float32)
coef = model_df["coefficient"].to_numpy(dtype=np.float32)

score_estimate = X @ coef

est_df = pd.DataFrame({"recipe_slug": ratings["recipe_slug"], "score_estimate": score_estimate})
with open("estimates.tsv", "w", newline="", encoding="utf-8") as f:
    est_df.to_csv(f, sep="\t", index=False)

print(f"Wrote estimates.tsv with {len(est_df)} rows.")


Wrote estimates.tsv with 100 rows.


Submit "estimates.tsv" in Gradescope.

## Part 5: LinUCB Bounds

Calculate the upper bounds of LinUCB using data corresponding to trying every recipe once and receiving the rating in "ratings.tsv" as the reward.
Keep the ridge regression regularization parameter at 1, and set LinUCB's $\alpha$ parameter to 2.
Save these upper bounds to a file "bounds.tsv" with columns recipe_slug and score_bound.

In [ ]:
# YOUR CHANGES HERE



import pandas as pd
import numpy as np

ratings  = pd.read_csv("ratings.tsv",  sep="\t", dtype={"recipe_slug": str})
features = pd.read_csv("features.tsv", sep="\t", dtype={"recipe_slug": str}).set_index("recipe_slug")

features = features.apply(pd.to_numeric, errors="coerce").fillna(0.0)
if "bias" in features.columns:
    features["bias"] = 1.0
else:
    features.insert(0, "bias", 1.0)

X = features.reindex(ratings["recipe_slug"]).values.astype(np.float64)   
y = ratings["rating"].to_numpy(dtype=np.float64)                         

lam = 1.0     
alpha = 2.0   

A = X.T @ X + lam * np.eye(X.shape[1], dtype=X.dtype)   
b = X.T @ y                                             

theta = np.linalg.solve(A, b)                           

estimate = X @ theta                                    

Z = np.linalg.solve(A, X.T)                             
var_component = np.sqrt(np.sum(X * Z.T, axis=1))        

score_bound = estimate + alpha * var_component          

out = pd.DataFrame({"recipe_slug": ratings["recipe_slug"], "score_bound": score_bound})
with open("bounds.tsv", "w", newline="", encoding="utf-8") as f:
    out.to_csv(f, sep="\t", index=False)

print(f"Wrote bounds.tsv with {len(out)} rows.")


Wrote bounds.tsv with 100 rows.


Submit "bounds.tsv" in Gradescope.

## Part 6: Make Online Recommendations

Implement LinUCB to make 100 recommendations starting with no data and using the same parameters as in part 5.
One recommendation should be made at a time and you can break ties arbitrarily.
After each recommendation, use the rating from part 1 as the reward to update the LinUCB data.
Record the recommendations made in a file "recommendations.tsv" with columns "recipe_slug", "score_bound", and "reward".
The rows in this file should be in the same order as the recommendations were made.

In [9]:
# YOUR CHANGES HERE



import pandas as pd
import numpy as np

ratings  = pd.read_csv("ratings.tsv",  sep="\t", dtype={"recipe_slug": str})
features = pd.read_csv("features.tsv", sep="\t", dtype={"recipe_slug": str}).set_index("recipe_slug")

features = features.apply(pd.to_numeric, errors="coerce").fillna(0.0)
if "bias" in features.columns:
    features["bias"] = 1.0
else:
    features.insert(0, "bias", 1.0)

X = features.reindex(ratings["recipe_slug"]).values.astype(np.float64)   
y = ratings["rating"].to_numpy(dtype=np.float64)                         
lam = 1.0     
alpha = 2.0   

A = X.T @ X + lam * np.eye(X.shape[1], dtype=X.dtype)   
b = X.T @ y                                             

theta = np.linalg.solve(A, b)                           

estimate = X @ theta                                    

Z = np.linalg.solve(A, X.T)                             
var_component = np.sqrt(np.sum(X * Z.T, axis=1))        

score_bound = estimate + alpha * var_component          

out = pd.DataFrame({"recipe_slug": ratings["recipe_slug"], "score_bound": score_bound})
with open("bounds.tsv", "w", newline="", encoding="utf-8") as f:
    out.to_csv(f, sep="\t", index=False)

print(f"Wrote bounds.tsv with {len(out)} rows.")


Wrote bounds.tsv with 100 rows.


Submit "recommendations.tsv" in Gradescope.

## Part 7: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


Submit "acknowledgments.txt" in Gradescope.

In [10]:
from pathlib import Path

ACK_FILE = Path("acknowledgments.txt")
ACK_TEXT = "My dad helped me with this"

def main():
    ACK_FILE.write_text(ACK_TEXT)
    print(f"Saved acknowledgments to: {ACK_FILE.resolve()}")

if __name__ == "__main__":
    main()

Saved acknowledgments to: /workspaces/dx704-project-04/acknowledgments.txt


## Part 8: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.


Submit "project.ipynb" in Gradescope.